# Исследование рынка общественного питания г.Москвы

**Автор** Пинчук Ольга

## 1. Загрузка данных и подготовка к анализу

In [83]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import requests
import json
from io import BytesIO


In [84]:
try:
    rest_data = pd.read_csv(f'datasets/rest_data.csv')  # Локальный путь
except:
    rest_data = pd.read_csv(f'/datasets/rest_data.csv')  # Серверный путь
display(rest_data.info())
display(rest_data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15366 entries, 0 to 15365
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           15366 non-null  int64 
 1   object_name  15366 non-null  object
 2   chain        15366 non-null  object
 3   object_type  15366 non-null  object
 4   address      15366 non-null  object
 5   number       15366 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 720.4+ KB


None

,id,object_name,chain,object_type,address,number
0,151635,СМЕТАНА,нет,кафе,"город Москва, улица Егора Абакумова, дом 9",48
1,77874,Родник,нет,кафе,"город Москва, улица Талалихина, дом 2/1, корпус 1",35
2,24309,Кафе «Академия»,нет,кафе,"город Москва, Абельмановская улица, дом 6",95
3,21894,ПИЦЦЕТОРИЯ,да,кафе,"город Москва, Абрамцевская улица, дом 1",40
4,119365,Кафе «Вишневая метель»,нет,кафе,"город Москва, Абрамцевская улица, дом 9, корпус 1",50


In [85]:
rest_data.columns

Index(['id', 'object_name', 'chain', 'object_type', 'address', 'number'], dtype='object')

In [86]:
# Заменяем значения в chain и приводим к типу bool
rest_data = rest_data.replace({'chain':{'нет': False, 'да': True}})
rest_data['chain'].astype('bool')
rest_data['object_name'].astype('str')
rest_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15366 entries, 0 to 15365
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           15366 non-null  int64 
 1   object_name  15366 non-null  object
 2   chain        15366 non-null  bool  
 3   object_type  15366 non-null  object
 4   address      15366 non-null  object
 5   number       15366 non-null  int64 
dtypes: bool(1), int64(2), object(3)
memory usage: 615.4+ KB


In [87]:
rest_data['object_type'].unique()

array(['кафе', 'столовая', 'закусочная',
       'предприятие быстрого обслуживания', 'ресторан', 'кафетерий',
       'буфет', 'бар', 'магазин (отдел кулинарии)'], dtype=object)

In [88]:
rest_data.isna().sum()

id             0
object_name    0
chain          0
object_type    0
address        0
number         0
dtype: int64

In [89]:
rest_data['address'] = rest_data['address'].str.lower()
rest_data['object_name'] = rest_data['object_name'].str.lower()

In [90]:
display(rest_data['object_name'].unique())
rest_data['object_name'].value_counts()

array(['сметана', 'родник', 'кафе «академия»', ..., 'бар мята lounge',
       'мята lounge октябрьская', 'кальянная «мята lounge»'], dtype=object)

столовая                      321
кафе                          278
шаурма                        250
шоколадница                   158
kfc                           155
                             ... 
столовая гуп «мосгортранс»      1
пиццерия «алло пицца»           1
bizone антикафе                 1
сирень                          1
кальянная «мята lounge»         1
Name: object_name, Length: 9924, dtype: int64

In [91]:
# Убираем из object_name пробелы в начале и конце, знаки ковычек и слова содержащиеся в object_type

rest_data['object_name'].replace(r'^\s+|\s+$|«|»', '', regex=True, inplace = True)

for t in rest_data['object_type'].unique():
    rest_data['object_name'].replace(f'{t}\s+', '', regex=True, inplace = True)


display(rest_data['object_name'].unique())    
rest_data['object_name'].value_counts()

array(['сметана', 'родник', 'академия', ..., 'магазин готовой еды милти',
       'лаундж-мята lounge', 'мята lounge октябрьская'], dtype=object)

столовая                   324
кафе                       279
шаурма                     253
шоколадница                176
kfc                        176
                          ... 
хлеб & мясо                  1
школа 1580 кдп               1
шаурма, донер кебаб          1
фуза                         1
мята lounge октябрьская      1
Name: object_name, Length: 9332, dtype: int64

In [92]:
rest_data.duplicated().sum()

0

In [93]:
rest_data_duplic = rest_data.drop('id', axis =1)
rest_data_duplic.duplicated().sum()

86

### Результат предобработки

- Данные проверены и приведены типы в нужных столбцах,
- Данные проверены на наличие пропусков,
- Столбец object_type проверен на наличие пробелов в конце
- В столбце object_name убрали пробелы в начале и конце, знаки ковычек и слова содержащиеся в object_type
- Таблица проверена на наличие дубликатов

## 2. Анализ данных

### 2.1. Cоотношение видов объектов общественного питания по количеству.

In [94]:
df_ObjType = rest_data.groupby('object_type')['id'].count().reset_index().sort_values(by ='id', ascending =False)
fig = px.pie(df_ObjType, values='id', names='object_type')

fig.update_layout(
                  title="1. Cоотношение видов объектов общественного питания по кол-ву",
                  margin=dict(l=0, r=0, t=30, b=0),
                  height=350,
                  width=700)

fig.show()

- [картинка](img/img1.jpg)

### 2.2. Соотношение сетевых и несетевых заведений по количеству

In [95]:
df_chain = rest_data.groupby('chain')['id'].count().reset_index()
fig = px.pie(df_chain, values='id', names='chain')

fig.update_layout(
                  title="2. Соотношение сетевых и несетевых заведений по количеству",
                  margin=dict(l=0, r=0, t=30, b=0),
                  height=300,
                  width=700)

fig.show()


- [картинка](img/img2.jpg)

### 2.3. Для какого вида объекта общественного питания характерно сетевое распространение?

In [96]:
df_ObjType_ch = rest_data.groupby(['object_type','chain'])['id'].count().reset_index().sort_values(by ='id', ascending =False)
fig = px.bar(df_ObjType_ch, x='object_type', y='id', color='chain', barmode="group", labels= {'chain':'Сеть/Не сеть'})

fig.update_layout(
                  title="3.1 Соотношение сетей/не сетей по видам обьектов",
                  xaxis_title="Вид обьекта",
                  yaxis_title="Кол-во",
                  margin=dict(l=0, r=0, t=30, b=0),
                  height=600,
                  width=900)

fig.show()

- [картинка](img/img3.jpg)

In [97]:
                 
labels = df_ObjType_ch['object_type'].unique()
specs = [[{'type':'domain'}, {'type':'domain'}]]
fig = make_subplots(1, 2, specs=specs, subplot_titles=['Сеть', 'Не сеть'])
    
fig.add_trace(go.Pie(labels=labels, values=list(df_ObjType_ch.query('chain == True')['id']), name ='Сеть'), 1, 1)
fig.add_trace(go.Pie(labels=labels, values=list(df_ObjType_ch.query('chain == False')['id']), name ='Не сеть'), 1, 2)

fig.update_layout(
                  title="Соотношение сетей/не сетей по видам обьектов",
                  margin=dict(l=40, r=0, t=30, b=0),
                  height=600,
                  width=900)

fig.show()

- [картинка](img/img4.jpg)

### Выводы по графикам 1-3

1. В общем распространении преобладает формат кафе, далее, с отрывом почти в 2 раза, идут столовые и рестораны
2. Кол-во не сетевых заведений больше в 4 раза
3. Сетевое распространение преобладает среди предприятия быстрого питания, кафе, рестораны, но даже в этих категориях большую долю занимают не сетевые рестораны

### 2.4. Что характерно для сетевых заведений: много заведений с небольшим числом посадочных мест в каждом или мало заведений с большим количеством посадочных мест?

In [98]:
chain = rest_data.query('chain == True')
x_val = pd.Series(range(0,len(chain)))
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_val, y=chain['number'], mode='markers'))

fig.update_layout(
                  title="4. График распределения кол-ва посадочных мест к кол-ву заведений для сетей",
                  yaxis_title="Кол-во заведений",
                  xaxis_title="Кол-во посадочных мест",
                  margin=dict(l=0, r=20, t=30, b=0),
                  height=320,
                  width=900)
fig.show()

- [картинка](img/img5.jpg)

### 2.5. Cреднее количество посадочных мест. Какой вид предоставляет в среднем самое большое количество посадочных мест?


In [99]:
type_mean_number = rest_data.groupby('object_type')['number'].mean().reset_index().sort_values(by ='number', ascending =False)
type_mean_number = type_mean_number.rename(columns={'number': 'mean_number'})
fig = px.pie(type_mean_number, values='mean_number', names='object_type')

fig.update_layout(
                  title="5. Виды заведений по кол-ву посадочных мест",
                  margin=dict(l=0, r=0, t=30, b=0),
                  height=300,
                  width=700)

fig.show()

- [картинка](img/img6.jpg)

### Выводы графики 4 и 5

1. Среди сетевых заведений нет определенной корреляции между кол-вом заведений и посадочных мест в них
2. Больше всего посадочных мест в столовых и ресторанах

### 2.6. Выделяем улицу из адреса

In [100]:
rest_data['street'] = rest_data['address'].str.split(", ", n=3, expand=True)[1]
rest_data['city'] = rest_data['address'].str.split(", ", n=3, expand=True)[0]
rest_data.head()



,id,object_name,chain,object_type,address,number,street,city
0,151635,сметана,False,кафе,"город москва, улица егора абакумова, дом 9",48,улица егора абакумова,город москва
1,77874,родник,False,кафе,"город москва, улица талалихина, дом 2/1, корпус 1",35,улица талалихина,город москва
2,24309,академия,False,кафе,"город москва, абельмановская улица, дом 6",95,абельмановская улица,город москва
3,21894,пиццетория,True,кафе,"город москва, абрамцевская улица, дом 1",40,абрамцевская улица,город москва
4,119365,вишневая метель,False,кафе,"город москва, абрамцевская улица, дом 9, корпус 1",50,абрамцевская улица,город москва


In [101]:
# Найдем кол-во строк с некоректными данными
fail_city =rest_data[rest_data['city'].str.count(r'город') != 1]
print('Кол-во записей с некоретной записью города -',len(fail_city))
print('% записей с некоретной записью города - {:.2f}%'.format(len(fail_city)/len(rest_data)*100))

Кол-во записей с некоретной записью города - 67
% записей с некоретной записью города - 0.44%


Некоректных записей менее 1% - удаляем

In [102]:
rest_data = rest_data.query('id not in @fail_city.id')
len(rest_data)

15299

### 2.7. Топ-10 улиц по количеству объектов общественного питания. В каких районах Москвы находятся эти улицы?


Для получения района города воспульзуемся csv из открытых источников
Далее для получения координат района воспользуемся геокодером яндекс- это потребуется для дальнейшей отрисовки на карте

In [103]:
# Загружаем файл с соответствием улиц районам города
url = 'https://docs.google.com/spreadsheets/d/1UAjd1RfjqKeHpKxtp-HG6srW8L7RuPJSkzS1ZJuD3xQ/edit?usp=sharing'

id_ = url.split('/')[5]
file_name = f'https://docs.google.com/spreadsheets/d/{id_}/export?format=csv'
r = requests.get(file_name)
district_moscow = pd.read_csv(BytesIO(r.content))
#district_moscow = pd.read_csv('http://hubofdata.ru/storage/f/2013-10-22T09%3A25%3A27.793Z/mosgaz-streets.csv')
district_moscow.head()

,streetname,areaid,okrug,area,Unnamed: 4
0,Выставочный переулок,17,ЦАО,Пресненский район,NaN
1,улица Гашека,17,ЦАО,Пресненский район,NaN
2,Большая Никитская улица,17,ЦАО,Пресненский район,NaN
3,Глубокий переулок,17,ЦАО,Пресненский район,NaN
4,Большой Гнездниковский переулок,17,ЦАО,Пресненский район,NaN


In [104]:
district_moscow['streetname'] = district_moscow['streetname'].str.lower()
district_moscow.head()

,streetname,areaid,okrug,area,Unnamed: 4
0,выставочный переулок,17,ЦАО,Пресненский район,NaN
1,улица гашека,17,ЦАО,Пресненский район,NaN
2,большая никитская улица,17,ЦАО,Пресненский район,NaN
3,глубокий переулок,17,ЦАО,Пресненский район,NaN
4,большой гнездниковский переулок,17,ЦАО,Пресненский район,NaN


In [105]:
# Подключаемся к геокодеру Яндекс
key = 'd6e49dce-d137-47b5-bce7-c610f937c439'
key2 = 'a080798a-c68a-47e8-864d-d40c7f3c8a94' #запасной ключь в случае исчерпания лимита подключений
params = {'format': 'json'}

# Функция для получения координат улицы

def yamap(address):
    url = 'https://geocode-maps.yandex.ru/1.x/?apikey='+ key2 + '&geocode=' + address
    
    response = requests.get(url, params=params)
    
     #получим координаты точки
    coord = response.json()['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['Point']['pos']
    
    return coord

In [106]:
rest_data = rest_data.merge(district_moscow[['streetname','okrug', 'area']], left_on='street', right_on='streetname')
rest_data.head()

,id,object_name,chain,object_type,address,number,street,city,streetname,okrug,area
0,151635,сметана,False,кафе,"город москва, улица егора абакумова, дом 9",48,улица егора абакумова,город москва,улица егора абакумова,СВАО,Ярославский Район
1,193477,grill-house,False,закусочная,"город москва, улица егора абакумова, дом 14а",0,улица егора абакумова,город москва,улица егора абакумова,СВАО,Ярославский Район
2,196756,пекарня,False,кафетерий,"город москва, улица егора абакумова, дом 14а",12,улица егора абакумова,город москва,улица егора абакумова,СВАО,Ярославский Район
3,77874,родник,False,кафе,"город москва, улица талалихина, дом 2/1, корпус 1",35,улица талалихина,город москва,улица талалихина,ЦАО,Таганский район
4,77874,родник,False,кафе,"город москва, улица талалихина, дом 2/1, корпус 1",35,улица талалихина,город москва,улица талалихина,ЮВАО,Нижегородский район


In [107]:
group = rest_data.groupby(['area','street','city']).agg({'id':'count', 'number': 'sum'}).reset_index()


In [108]:
group_area = rest_data.groupby(['area','street', 'city']).agg({'id':'count', 'number': 'sum'}).reset_index()
group_area = group_area.rename(columns={'id': 'count_id', 'number': 'sum_number'})
group_street = group_area.groupby(['street']).agg({'count_id':'sum', 'sum_number': 'sum'}).reset_index()
group_street = group_street.reset_index(drop =True)

top10str = group_street.sort_values(by ='count_id', ascending =False).head(10)
top10str

,street,count_id,sum_number
1014,проспект мира,1428,89530
392,варшавское шоссе,1288,68104
671,ленинский проспект,1176,74640
1018,профсоюзная улица,1092,50682
485,дмитровское шоссе,700,44233
457,город зеленоград,696,51150
668,ленинградский проспект,684,36040
1012,проспект вернадского,660,44200
669,ленинградское шоссе,600,26808
585,каширское шоссе,448,24684


In [109]:
fig = px.bar(top10str, x='street', y='count_id')

fig.update_layout(
                  title="7. Топ 10 улиц по кол-ву заведений",
                  xaxis_title="Улица",
                  yaxis_title="Кол-во заведений",
                  margin=dict(l=0, r=0, t=30, b=0),
                  height=400,
                  width=900)

fig.show()

- [картинка](img/img7.jpg)

In [110]:

top10str_area = group_area.query('street in @top10str.street ')
top10str_area['city_area'] =top10str_area['city']+', '+ top10str_area['area']
coordin = pd.DataFrame()
coordin['city_area'] =top10str_area['city_area'].unique()
coordin['area'] = top10str_area['area'].unique()
coordin.head()


C:\Users\osia\AppData\Local\Temp\ipykernel_13536\949044282.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,city_area,area
0,"город москва, Академический район",Академический район
1,"город москва, Алексеевский район",Алексеевский район
2,"город москва, Бескудниковский Район",Бескудниковский Район
3,"город москва, Войковский Район",Войковский Район
4,"город москва, Головинский район",Головинский район


In [111]:
coordin['coord_area'] = coordin['city_area'].apply(lambda row: yamap(row))
print('Кол-во записей:', len(coordin))
coordin.reset_index(drop=True)
coordin.head()

Кол-во записей: 48


,city_area,area,coord_area
0,"город москва, Академический район",Академический район,37.573124 55.689314
1,"город москва, Алексеевский район",Алексеевский район,37.650172 55.812954
2,"город москва, Бескудниковский Район",Бескудниковский Район,37.554717 55.866627
3,"город москва, Войковский Район",Войковский Район,37.497342 55.827247
4,"город москва, Головинский район",Головинский район,37.51061 55.846765


In [112]:
top10str_area = top10str_area.merge(coordin[['area', 'coord_area']], on='area')
top10str_area['lat'] = top10str_area['coord_area'].str.split(" ", n=2, expand=True)[1].astype('float64')
top10str_area['lon'] = top10str_area['coord_area'].str.split(" ", n=2, expand=True)[0].astype('float64')
top10str_area.head()

,area,street,city,count_id,sum_number,city_area,coord_area,lat,lon
0,Академический район,профсоюзная улица,город москва,182,8447,"город москва, Академический район",37.573124 55.689314,55.689314,37.573124
1,Алексеевский район,проспект мира,город москва,204,12790,"город москва, Алексеевский район",37.650172 55.812954,55.812954,37.650172
2,Бескудниковский Район,дмитровское шоссе,город москва,100,6319,"город москва, Бескудниковский Район",37.554717 55.866627,55.866627,37.554717
3,Войковский Район,ленинградское шоссе,город москва,100,4468,"город москва, Войковский Район",37.497342 55.827247,55.827247,37.497342
4,Головинский район,ленинградское шоссе,город москва,100,4468,"город москва, Головинский район",37.51061 55.846765,55.846765,37.510610


In [113]:

map_center = {'lat': (top10str_area['lat'].max()+top10str_area['lat'].min())/2, 'lon': (top10str_area['lon'].max()+top10str_area['lon'].min())/2}
fig = px.scatter_mapbox(top10str_area, lat='lat', lon='lon', 
                        mapbox_style='open-street-map', center=map_center, zoom=8,
                        text=top10str_area.area +', '+ top10str_area.street, size='count_id', 
                        color='count_id')

fig.show()

- [картинка](img/img8.jpg)

### Вывод

Больше всего заведений сосредоточено в Зеленограде- отделенный округ г.Москвы, в основном изолирован от остальной москвы, но является центром для окружающих его районов подмосковья.

Далее идет пр.Мира (несколько районов)- здесь сосредоточены ВДНХ, гостинеца Космос, Рижский вокзал- одни из значемых туристических обьектов г. Москвы.

### 2.8. Найти число улиц с одним объектом общественного питания. В каких районах Москвы находятся эти улицы?


In [114]:
teilObj = group_street.query('count_id == 1')
#teilObj2 = rest_data.query('street in @teilObj.street')
teilObj_area = group_area.query('street in @teilObj.street')
teilObj_area['city_area'] =teilObj_area['city']+', '+ teilObj_area['area']
teilObj_area.head()

C:\Users\osia\AppData\Local\Temp\ipykernel_13536\2048738421.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,area,street,city,count_id,sum_number,city_area
13,Алексеевский район,1-й рижский переулок,город москва,1,112,"город москва, Алексеевский район"
17,Алексеевский район,зубарев переулок,город москва,1,47,"город москва, Алексеевский район"
18,Алексеевский район,кулаков переулок,город москва,1,150,"город москва, Алексеевский район"
19,Алексеевский район,кучин переулок,город москва,1,134,"город москва, Алексеевский район"
23,Алексеевский район,рижский проезд,город москва,1,0,"город москва, Алексеевский район"


In [115]:
len(teilObj_area['city_area'].unique())

94

In [116]:
teil_coordin =pd.DataFrame()
teil_coordin['city_area'] =top10str_area['city_area'].unique()
teil_coordin['area'] = top10str_area['area'].unique()
teil_coordin.head()

,city_area,area
0,"город москва, Академический район",Академический район
1,"город москва, Алексеевский район",Алексеевский район
2,"город москва, Бескудниковский Район",Бескудниковский Район
3,"город москва, Войковский Район",Войковский Район
4,"город москва, Головинский район",Головинский район


In [117]:
#ячейка долго отрабатывает из-за подключения к геокодеру
teil_coordin['coord_area'] = teil_coordin['city_area'].apply(lambda row: yamap(row))
print('Кол-во записей:', len(teil_coordin))
teil_coordin.reset_index(drop=True)
teil_coordin.head()

Кол-во записей: 48


,city_area,area,coord_area
0,"город москва, Академический район",Академический район,37.573124 55.689314
1,"город москва, Алексеевский район",Алексеевский район,37.650172 55.812954
2,"город москва, Бескудниковский Район",Бескудниковский Район,37.554717 55.866627
3,"город москва, Войковский Район",Войковский Район,37.497342 55.827247
4,"город москва, Головинский район",Головинский район,37.51061 55.846765


In [118]:
teilObj_area = teilObj_area.merge(teil_coordin[['area', 'coord_area']], on='area')
teilObj_area['lat'] = teilObj_area['coord_area'].str.split(" ", n=2, expand=True)[1].astype('float64')
teilObj_area['lon'] = teilObj_area['coord_area'].str.split(" ", n=2, expand=True)[0].astype('float64')
teilObj_area.head()

,area,street,city,count_id,sum_number,city_area,coord_area,lat,lon
0,Алексеевский район,1-й рижский переулок,город москва,1,112,"город москва, Алексеевский район",37.650172 55.812954,55.812954,37.650172
1,Алексеевский район,зубарев переулок,город москва,1,47,"город москва, Алексеевский район",37.650172 55.812954,55.812954,37.650172
2,Алексеевский район,кулаков переулок,город москва,1,150,"город москва, Алексеевский район",37.650172 55.812954,55.812954,37.650172
3,Алексеевский район,кучин переулок,город москва,1,134,"город москва, Алексеевский район",37.650172 55.812954,55.812954,37.650172
4,Алексеевский район,рижский проезд,город москва,1,0,"город москва, Алексеевский район",37.650172 55.812954,55.812954,37.650172


In [119]:
teilObj_area2 = teilObj_area.groupby(['area','lat','lon'])['count_id'].sum().reset_index()

In [120]:
map_center = {'lat': (teilObj_area2['lat'].max()+teilObj_area2['lat'].min())/2, 'lon': (teilObj_area2['lon'].max()+teilObj_area2['lon'].min())/2}
fig = px.scatter_mapbox(teilObj_area2, lat='lat', lon='lon', 
                        mapbox_style='open-street-map', center=map_center, zoom=8,
                        text='area', size='count_id', 
                        color='count_id')

fig.show()

- [картинка](img/img9.jpg)

### Вывод

Больше всего улиц с 1 заведением сосредоточены в районе Марьина Роща, их там 16шт. В данном районе расположены Савеловский рынок (самый крупный ИТ рынок) и рижский вокзал. Эти крупные обьекты являются точками притяжения и работы иммигрантов среди которых популярны небольшие и не дорогие заведения.

### 2.9. Изучение распределения количества посадочных мест для улиц с большим количеством объектов общественного питания. Какие закономерности можно выявить?

In [121]:
fig = px.bar(top10str.sort_values(by ='sum_number', ascending = False), x='street', y='sum_number')

fig.update_layout(
                  title="9. Топ 10 улиц по кол-ву посадочных мест",
                  xaxis_title="Улица",
                  yaxis_title="Кол-во посадочных мест",
                  margin=dict(l=0, r=0, t=30, b=0),
                  height=400,
                  width=900)

fig.show()

- [картинка](img/img10.jpg)

In [122]:

map_center = {'lat': (top10str_area['lat'].max()+top10str_area['lat'].min())/2, 'lon': (top10str_area['lon'].max()+top10str_area['lon'].min())/2}
fig = px.scatter_mapbox(top10str_area, lat='lat', lon='lon', 
                        mapbox_style='open-street-map', center=map_center, zoom=8,
                        text=top10str_area.area +', '+ top10str_area.street, size='sum_number', 
                        color='sum_number')
fig.update_layout(
                  title="9.1 Районы для топ 10 улиц по кол-ву посадочных мест",
                  margin=dict(l=0, r=0, t=30, b=0),
                  height=400,
                  width=900)
fig.show()

- [картинка](img/img11.jpg)

In [123]:
print('Корреляция между кол-вом заведений и кол-вом посадочных мест')
top10str.corr()

Корреляция между кол-вом заведений и кол-вом посадочных мест


,count_id,sum_number
count_id,1.000000,0.929705
sum_number,0.929705,1.000000


### Вывод

Кол-во посадочных мест на улице прямо связано с кол-во заведений на ней, чем болье заведений, тем больше посадочных мест. КОрреляция между этими параметрами очень высокая (0,93).

## Общий вывод

1. В общем распространении преобладает формат кафе, далее, с отрывом почти в 2 раза, идут столовые и рестораны
2. Кол-во не сетевых заведений больше в 4 раза
3. Сетевое распространение преобладает среди предприятия быстрого питания, кафе, рестораны, но даже в этих категориях большую долю занимают не сетевые рестораны
4. Среди сетевых заведений нет определенной корреляции между кол-вом заведений и посадочных мест в них
5. В среднем больше всего посадочных мест в столовых(130 мест) и ресторанах (96 мест)
6. Больше всего заведений сосредоточено в Зеленограде- отделенный округ г. Москвы, в основном изолирован от остальной Москвы, но является центром для окружающих его районов Подмосковья. Далее идет пр. Мира (несколько районов)- здесь сосредоточены ВДНХ, гостиница Космос, Рижский вокзал- одни из значимых туристических объектов г. Москвы.
7. Больше всего улиц с 1 заведением сосредоточены в районе Марьина Роща, их там 16шт.В данном районе расположены Савеловский рынок (самый крупный ИТ рынок) и рижский вокзал. Эти крупные объекты являются точками притяжения и работы иммигрантов среди которых популярны небольшие и не дорогие заведения.
8. Распределение кол-ва посадочных мест по улицам прямо связано с кол-во заведений на ней, чем больше заведений, тем больше посадочных мест. Корреляция между этими параметрами очень высокая (0,93)
